In [4]:
from csg.core import CSG
from csg.geom import Vertex, Vector, Plane, Polygon
import meshplot as mp
import numpy as np
import geo_vase

def plotCsg(csg):
    verts, polys, count = csg.toVerticesAndPolygons()
    triangles = []

    tris = []
    # polys are always convex (apparently...)
    for poly in polys:
        vlen = len(poly)
        for n in range(1,vlen-1):
            tris.append(
                [
                    poly[0],
                    poly[n%vlen],
                    poly[(n+1)%vlen],
                ]
            )
    mp.plot(np.array(verts), np.array(tris), shading={"point_size": 0.1})

gv = geo_vase.GeoVase(
    rows=7,
    sides=5,
    spiral=100
)

# Nice --- easy to convert from our mesh to CSG mesh.
ps = CSG.fromPolygons([Polygon([Vertex(gv.verts[v].tolist()) for v in reversed(f)]) for f in gv.faces])
b = CSG.fromPolygons([Polygon([Vertex(gv.verts[v].tolist()) for v in reversed(f)]) for f in gv.faces])
b.rotate([0, 1, 0], 30)

plotCsg(ps)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.003524…

In [11]:
def frameCube(center, frame, scale):
    n, u, v = frame
    verts = []
    
    for i in range(8):
        a = (2 * bool(i & 1) - 1)
        b = (2 * bool(i & 2) - 1)
        c = (2 * bool(i & 4) - 1)

        verts.append(Vertex((center - (n*a + u*b + v*c)*scale).tolist()))
    polys = []
    cubeFaces = [
        [[0, 4, 6, 2], [-1, 0, 0]],
        [[1, 3, 7, 5], [+1, 0, 0]],
        [[0, 1, 5, 4], [0, -1, 0]],
        [[2, 6, 7, 3], [0, +1, 0]],
        [[0, 2, 3, 1], [0, 0, -1]],
        [[4, 5, 7, 6], [0, 0, +1]]
    ]
    
    for f,_ in cubeFaces:
        polys.append(Polygon([ verts[v] for v in f]))

    return CSG.fromPolygons(polys)



gv = geo_vase.GeoVase(
    rows=11,
    sides=5,
    spiral=50
)

c = CSG.cube(center=[0,0.65,0], radius=[10, 0.55, 10])

for i, vert in enumerate(gv.verts):
    vnorms = gv.vnorms[i]
    n, u, v = vnorms
    r = 0.05
    q = 10
    c = c - frameCube(vert - n*q, vnorms, q + r)


plotCsg(c)
verts, polys, count = c.toVerticesAndPolygons()
print({
    "vertices": verts,
    "faces": [[x for x in reversed(p)] for p in polys]
})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001235…

{'vertices': [(-0.4129465161799999, 0.3159800518000002, 0.22515018360000005), (-0.5637433963499999, 0.49862433530000017, -0.011186974399999938), (-0.5539391007399999, 0.47747092170000016, -0.031690533299999935), (-0.35629767992999994, 0.14726742350000022, -0.07303853219999978), (-0.3250792387199999, 0.10000000000000009, -0.06066617569999999), (-0.2943829884899999, 0.10000000000000009, 0.1311718206000001), (0.2536837739000002, 1.1470029011000002, -0.41178261988999987), (0.11583925500000003, 1.2, -0.3475726268699999), (0.33923233090000005, 1.2, -0.16012357929999999), (0.3788290484000001, 1.1672852024000002, -0.2379339399699999), (0.2571151288000002, 1.2, 0.27314821160000013), (0.34335323800000017, 1.1672852024000002, 0.2867622040000002), (0.47002114130000017, 1.1470029011000002, 0.11401977870000013), (0.36635751010000006, 1.2, 0.002763829800000117), (-0.18032644229999995, 1.2, 0.3289384580000001), (-0.16662507709999996, 1.1672852024000002, 0.41516272870000015), (0.03680526680000007, 1.14